In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
gtes = "gtes03ene.xls"
goi = "goi03ene.xls"
gom = "gom03ene.xls"
gef = "gef03ene.xls"
cuentas=[7,10,11,12,16,17]
bdgtes = pd.read_excel('../data/'+gtes)
bdgoi = pd.read_excel('../data/'+goi)
bdgom = pd.read_excel('../data/'+gom)
bdgef = pd.read_excel('../data/'+gef)

bdBruto= pd.concat([bdgtes,bdgoi,bdgom,bdgef],ignore_index=True)
#bd.describe()


In [2]:
#bdBruto.columns

In [3]:
bdReducida=bdBruto.loc[:,('cve_tipo_comprob','glosa_reng','cve_debe_haber','monto_mo',
           'cod_moneda', 'cod_movimiento', 'nom_movimiento',
           'monto_mn','glosa_comprob','nro_comprob','cod_mayor')]
#bdReducida

In [4]:
dict = {'cve_tipo_comprob':'tipo','glosa_reng':'glosaRenglon','cve_debe_haber':'dh','monto_mo':'mo',
           'cod_moneda':'moneda', 'cod_movimiento':'codMov', 'nom_movimiento':'nomMov',
           'monto_mn':'mn','glosa_comprob':'glosa','nro_comprob':'comprobante','cod_mayor':'mayor'}
bdReducida.rename(columns=dict,
          inplace=True)
bdReducida.shape

(5310, 11)

In [5]:
#bd['mayor'].unique()

#bdFiltro1=bdReducida[bdReducida['mayor'].isin([7,10,11,16,17])]
bdFiltro1=bdReducida[bdReducida['mayor'].isin(cuentas)]

In [6]:
#bdFiltro1.loc[:,("comprobante","mayor")]
bdFiltro1["comprobante"].unique()
#bdFiltro1.sort_values(by="comprobante", ascending=True)

array([1988644, 1988660, 1988673, 1988688, 1988768, 1988773, 1988781,
       1023054, 1023055,   28345, 1023056,   21758, 1023069, 1023070,
       1023073, 1746414, 1746970, 1023046, 1746866, 1746530, 1746531,
       1746532, 1746533, 1746534, 1746535, 1746536, 1746737, 1746740,
       1023071, 1746862, 1746864, 1746868,    5315], dtype=int64)

In [7]:
#bd[bd['comprobante'].isin(bdFiltro1["comprobante"])].sort_values(by="comprobante", ascending=True).tail(60)
bd=bdReducida[bdReducida["comprobante"].isin(bdFiltro1["comprobante"])].sort_values(by=["comprobante","dh"], ascending=True)
bd = bd.reset_index(drop=True)
#bd.head(60)

In [8]:
#list(bd.loc[55,["glosa"]])

In [9]:
#bd.to_excel("ux.xlsx")

In [10]:
bd=bd.drop_duplicates()
bd = bd.reset_index(drop=True)
#bd.head(60)

In [11]:
#bd["comprobante"].unique()

In [12]:
def llenarData(concepto,a,b):
    if concepto !="Neteo":    
        print('dentro ',pares["Comprobante"][a])
        data2.append({
                 "Tipo":pares["Tipo"][a],"Comprobante":pares["Comprobante"][a],"Concepto":concepto,
                "Mayor1":pares["Mayor"][a],"Mayor2":pares["Mayor"][b],"codMov":pares["codMov"][b],
                "Movimiento":pares["Movimiento"][b],"MontoUSD":round(pares["MN"][a]/6.86,2),
                "Detalle":pares["GlosaDetalle"][b],"Glosa":pares["Glosa"][a]
                })
    else:
        data2.append({
                "Tipo":pares["Tipo"][a],"Comprobante":pares["Comprobante"][a],"Concepto":concepto,
                "Mayor1":pares["Mayor"][a],"Mayor2":pares["Mayor"][b],"codMov":pares["codMov"][b],
                "Movimiento":pares["Movimiento"][b],"MontoUSD":0, 
                "Detalle":("±", round(pares["MN"][a]/6.86,2)),"Glosa":pares["Glosa"][a]
                })

def dv(concepto,suma):
    data2.append({''
                 "Tipo":aux2["tipo"][0],"Comprobante":aux2["comprobante"][0],"Concepto":concepto,
                "Mayor1":"--","Mayor2":"--","codMov":"--",
                "Movimiento":"--","MontoUSD":round(suma,2),
                "Detalle":"--","Glosa":aux2["glosa"][0]
                })
    

In [23]:
aux1=bd["comprobante"].unique()
consolidado=pd.DataFrame()

for q in range(len(aux1)):
    aux2 = bd[bd["comprobante"]==aux1[q]]
    aux2=aux2.reset_index(drop=True)
    data1=[]
    columnas=["Tipo","Comprobante","Concepto","Mayor","codMov","Movimiento","MN","Glosa","GlosaDetalle"]
    print("Comprobante ",q)
    print(aux2["comprobante"][0])
    if aux2["tipo"][0]=='V' or aux2["tipo"][0]=='D': ####### los ajustes por arbitraje y revalorizacion ######
        data2=[]
        columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes
        sumaDebe=0
        sumaHaber=0
        for i in aux2.index:
            if aux2["mayor"][i] in cuentas and aux2["dh"][i]=='D':
                sumaDebe=sumaDebe+aux2["mn"][i]/6.86
            elif aux2["mayor"][i] in cuentas and aux2["dh"][i]=='H':
                sumaHaber=sumaHaber+aux2["mn"][i]/6.86
        suma=sumaDebe-sumaHaber
        if suma>0:
            dv("Ingreso",suma)
            unRegistro=pd.DataFrame(data2,columns=columnas)
            consolidado=consolidado.append(unRegistro, ignore_index=True)
        else:
            dv("Egreso",-suma)
            unRegistro=pd.DataFrame(data2,columns=columnas)
            consolidado=consolidado.append(unRegistro, ignore_index=True)
    else: # este el caso normal digamos ##############################################################
        #print ("menos Tipo D o V")
        print("Hay duplicado: ",aux2.duplicated(subset=['mn']).any())
        
        if aux2.duplicated(subset=['mn']).any()==True:
            aux3=aux2.duplicated(subset=['mn'],keep=False)
            aux4=aux3.index[aux3==True].tolist()
            print(aux4)
            for x in range(len(aux4)):

                data1.append({
                    "Tipo":aux2["tipo"][aux4[x]],"Comprobante":aux2["comprobante"][aux4[x]],"Concepto":aux2["dh"][aux4[x]],
                    "Mayor":aux2["mayor"][aux4[x]],"codMov":aux2["codMov"][aux4[x]],"Movimiento":aux2["nomMov"][aux4[x]],
                    "MN":aux2["mn"][aux4[x]],"Glosa":aux2["glosa"][aux4[x]],"GlosaDetalle":aux2["glosaRenglon"][aux4[x]],
                            })
                pares=pd.DataFrame(data1,columns=columnas)

        else:
            print('aux2')
            aux2= aux2.sort_values(by='mayor') #, key=cuentas)
            aux2=aux2.reset_index(drop=True)
            print(aux2)
            
            for x in range(len(aux2)):
                #print(aux[x],"=== ",aux2["mn"][aux4[x]])
                data1.append({
                    "Tipo":aux2["tipo"][x],"Comprobante":aux2["comprobante"][x],"Concepto":aux2["dh"][x],
                    "Mayor":aux2["mayor"][x],"codMov":aux2["codMov"][x],"Movimiento":aux2["nomMov"][x],
                    "MN":aux2["mn"][x],"Glosa":aux2["glosa"][x],"GlosaDetalle":aux2["glosaRenglon"][x],
                            })
                pares=pd.DataFrame(data1,columns=columnas)

            #print(aux2['mayor'][i],' ... ',aux2["comprobante"][i])
            #print('pares ',pares["Comprobante"][0])
        print(pares)
        data2=[]
        columnas=["Tipo","Comprobante","Concepto","Mayor1","Mayor2","codMov","Movimiento","MontoUSD","Detalle","Glosa"] #luego pones mas campos importantes

        if len(aux2)==2:
            print("largo de aux2: ",len(aux2))
            if pares["Mayor"][0] in cuentas and pares["Mayor"][1] in cuentas:
                #print("Posible Neteo")
                llenarData("Neteo",0,1)  ######## aqui ver para que el monto sea cero y que en el detalle salga el monto
                unRegistro=pd.DataFrame(data2,columns=columnas)
            else:     
                if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="D":
                    print('caso 1')
                    llenarData("Ingreso",0,1)        
                if pares["Mayor"][0] in cuentas and pares["Mayor"][1] not in cuentas and pares["Concepto"][0]=="H":
                    print('caso 2')
                    llenarData("Egreso",0,1)                   
                if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="D":
                    print('caso 3')
                    llenarData("Ingreso",1,0)                    
                if pares["Mayor"][1] in cuentas and pares["Mayor"][0] not in cuentas and pares["Concepto"][1]=="H":            
                    print('caso 4')
                    llenarData("Egreso",1,0)
        else:
            if len(aux2)==1:                
                print("entramos con 1")
                if pares["Concepto"][0]=="D":
                    print('caso 5')
                    llenarData("Ingreso",0,0)
                else:
                    print('caso 6')
                    llenarData("Egreso",0,0)
            else:
                print("mas de 2")
                

        unRegistro=pd.DataFrame(data2,columns=columnas)
        consolidado=consolidado.append(unRegistro, ignore_index=True)  

consolidado
#consolidado.to_excel("borrar1.xlsx")

Comprobante  0
5315
Hay duplicado:  True
[0, 1]
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S         5315        D     10    8236   
1    S         5315        H    606    5879   

                                          Movimiento      MN  \
0  CTA. 0D USD - PORTAFOLIO FPAH                 ...  915.12   
1  INTERESES S/DEPOSITOS A PLAZO EN EL EXTERIOR  ...  915.12   

                                               Glosa  GlosaDetalle  
0  ||REGISTRO DE INGRESO POR INTERESES AL FONDO D...           NaN  
1  ||REGISTRO DE INGRESO POR INTERESES AL FONDO D...           NaN  
largo de aux2:  2
caso 1
dentro  5315
Comprobante  1
21758
Comprobante  2
28345
Comprobante  3
1023046
Hay duplicado:  True
[0, 1]
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    S      1023046        D     16    6286   
1    S      1023046        H    499    7355   

                                          Movimiento      MN  \
0  TITULOS - PORTAFOLIO DE LIQUIDEZ              ...  266.85   
1  VALORA

  Tipo  Comprobante Concepto  Mayor  codMov  \
0    O      1988673        D      7    3491   
1    O      1988673        H    406     898   

                                          Movimiento      MN  \
0  DOLARES AMERICANOS EN TESORO PROPIO           ...  1372.0   
1  TGN CUENTA TRANSITORIA- M/E                   ...  1372.0   

                                               Glosa  GlosaDetalle  
0  DEPOSITO DE EFECTIVO, DEPOSITANTE: OLGA V. QUI...           NaN  
1  DEPOSITO DE EFECTIVO, DEPOSITANTE: OLGA V. QUI...           NaN  
largo de aux2:  2
caso 1
dentro  1988673
Comprobante  29
1988688
Hay duplicado:  True
[0, 1]
  Tipo  Comprobante Concepto  Mayor  codMov  \
0    O      1988688        D    403    3917   
1    O      1988688        H      7    3491   

                                          Movimiento          MN  \
0  BCO.GANADERO-CUENTA CORRIENTE Y DE ENCAJE     ...  27440000.0   
1  DOLARES AMERICANOS EN TESORO PROPIO           ...  27440000.0   

                  

,Tipo,Comprobante,Concepto,Mayor1,Mayor2,codMov,Movimiento,MontoUSD,Detalle,Glosa
0,S,5315,Ingreso,10,606,5879,INTERESES S/DEPOSITOS A PLAZO EN EL EXTERIOR ...,133.40,NaN,||REGISTRO DE INGRESO POR INTERESES AL FONDO D...
1,D,21758,Egreso,--,--,--,--,0.00,--,AJUSTE COMPLEMENTARIO POR REVALORIZACION SALDO...
2,V,28345,Ingreso,--,--,--,--,3008840.83,--,AJUSTE POR ARBITRAJE DE SALDOS DE ACTIVOS DE R...
3,S,1023046,Ingreso,16,499,7355,VALORACION TITULOS DE LAS RESERVAS INTERNACION...,38.90,NaN,||REVALORACION DE TITULOS A PRECIOS DE MERCADO...
4,S,1023054,Ingreso,11,606,1535,INTERESES OVERNIGHT CAPITAL TRABAJO ...,9797.70,NaN,'INTERESES DEVENGADOS SOBRE DEPOSITOS'||OVERNI...
5,S,1023056,Ingreso,17,606,6538,INTERESES SOBRE CUPONES PORTAFOLIO EURO ...,18062.16,NaN,||INTERESES DEVENGADOS DEL PORTAFOLIO GLOBAL E...
6,S,1023069,Egreso,10,485,5906,PROVISION COMISIONES Y GASTOS BANCARIOS EXTER...,177.39,NaN,||PAGO DE COMISIONES Y GASTOS BANCARIOS AL BAN...
7,S,1023070,Egreso,10,485,5906,PROVISION COMISIONES Y GASTOS BANCARIOS EXTER...,0.34,NaN,||PAGO DE COMISIONES Y GASTOS BANCARIOS AL BAN...
8,S,1023073,Ingreso,10,606,1533,INTERESES S/FONDOS PLAZO Y OVERNIGHT INVERSION...,0.33,NaN,INTERESES PERCIBIDOS EN||EN LA CUENTA 0D USD D...
9,G,1746414,Neteo,10,10,5896,NEW YORK - CTA. 3544020682001 ...,0.00,"(±, 768762.91)",TRANSFERENCIA DE FONDOS DE LA CUENTA CTR_SCB 3...
